# Cooling process with by-pass and recycling

Model: 
- Cooling coil with by-pass, thermal zone, mixing (recycling). 

Control:
- Indoor air temperature controls the total heat of the cooling coil.
- Indoor air humidity ratio controlls the by-pass ratio, so the damper system. (where it is known in advance).

In [1]:
import ipywidgets as wd
import matplotlib.pyplot as plt
import numpy as np
import psychro as psy

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)

In [2]:
import cool_dp as cc

## General Model with legend

<=5=================================5=========<==================
  mo      ||                        m                          ||
          5 (m-mo) =======1=======                             ||
          ||       ||  (1-β)m   ||                             ||
θo,φo=0=>[MX1]==1==/          [MX2]==3(=4)===>===F====[TZ]==5==||
 mo               |||           ||                     //      |
                  |===1=[CC]==2===                     sl      |
                  |     / \\   βm                      ||      |
                  |    t   sl                         [BL]<-mi |
                  |    |                                       |
                  |    |                                       |
                  |    |                                       |
                  |--------------------------------------------|<-φI
                       --------------------------[K]-----------|<-θI
               
LEGEND:
Inputs:
θo, φo      outdoor temperature & humidity ratio
θIsp, φIsp  indoor temperature & humidity ratio set points
mi          infiltration mass flow rate
Qsa, Qla    auxiliary sensible and latent loads [kW]

Parameters:
m           mass flow rate of dry air (depending on the problem)
α           fraction of fresh air
β           by-pass factir od cooling coil (depending on the problem)
UA          overall heat transfer coefficient

Elements (13 equations):
MX1         mixing box (2 equations)
CC          cooling coil (4 equations)
MX2         mixing box (2 equations)
TZ          thermal zone (2 equations)
BL          building (2 equations)
Kθ          indoor temperature controller (1 equation)
F           fan 

Outputs (13 unknowns):
0, ..., 3   temperature and humidity ratio (8 unknowns)
Qt, Qs, Ql  total, sensible and latent heat of CC (3 unknowns)
Qs, Ql      sensible and latent heat of TZ (2 unknowns)

### General parameters

In [3]:
# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat

θI, φI = 24, 0.6            # °C, %, indoor air
θo, φo = 32, 0.5           # °C, %, outdoor air
mo = 1.00                  # kg/s, outdoor (fresh) air
US, mi = 675, 0.7          # W/K, kg/s, overall conductivity, infiltration
Qsa, Qla = 17000, 2000    # W, W, sensible, latent auxiliary heat

### Estimation of the values of building characteristic.

In [4]:
wo = psy.w(θo, φo)
wI = psy.w(θI, φI)

QsTZ = (US + mi * c) * (θo - θI) + Qsa  # W, sensible load thermal zone
QlTZ = mi * l * (wo - wI) + Qla         # W, latent load thermal zone

### Air Handling unit object (AHU)

In [5]:
Kθ = 1e10
Kw = 0     # reheater switched off
β = 0.1    # initial guess
 
m, mo =3.5, 1
θo, φo = 32, 0.5
θIsp, φIsp = 24, 0.6

mi = 0.7   # kg/s
UA = 675  # W/K
Qsa, Qla = 17000, 2000  # W

parameters = m, mo, β, Kθ, Kw
inputs = θo, φo, θIsp, φIsp, mi, UA, Qsa, Qla

## Problem 1: The supply air mass flow rate is 3.50 kg/s (beta unknown).

In [6]:
cool1 = cc.MxCcTzBl(parameters, inputs)
wd.interact(cool1.VBP_wd, value='φI', sp=(0.2, 0.7), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500))

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.7, description='sp', max=0.7,…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

## Problem 2: The by-pass ratio is zero (supply mass flow rate unknown).

In [7]:
β = 0
parameters = m, mo, β, Kθ, Kw
cool1 = cc.MxCcTzBl(parameters, inputs)
wd.interact(cool1.m_ls, value='φI', sp=(0.2, 0.7))

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.44999999999999996, descriptio…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

## Problem 3: The by-pass ratio is 0.16 (supply mass flow rate unknown).

In [8]:
β = 0.16
parameters = m, mo, β, Kθ, Kw
cool1 = cc.MxCcTzBl(parameters, inputs)
wd.interact(cool1.m_ls, value='φI', sp=(0.2, 0.7))

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.44999999999999996, descriptio…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>